In [6]:
from flask import Flask, request, jsonify
from pyperiscope import Pilot
from collections import OrderedDict
import uuid
import threading
import os
from typing import Dict, Optional

app = Flask(__name__)

class ProcessManager:
    def __init__(self, max_size: int = 50):
        self.processes: OrderedDict[str, Dict] = OrderedDict()
        self.max_size = max_size
        self.lock = threading.Lock()
        self.current_process: Optional[Dict] = None
        
    def add_process(self, guid: str, state: Dict) -> None:
        with self.lock:
            if len(self.processes) >= self.max_size:
                self.processes.popitem(last=False)
            self.processes[guid] = state
            print(f"Added process with GUID: {guid}")  # Debugging
            print(f"Processes in manager: {self.processes}")  # Debugging
            
    def get_process(self, guid: str) -> Optional[Dict]:
        with self.lock:
            return self.processes.get(guid)
    
    def set_current_process(self, process: Optional[Dict]) -> None:
        with self.lock:
            self.current_process = process
            
    def get_current_process(self) -> Optional[Dict]:
        with self.lock:
            return self.current_process

process_manager = ProcessManager()

@app.route('/run', methods=['POST'])
def run_notebook():
    current_process = process_manager.get_current_process()
    if current_process:
        return jsonify({
            'status': 'busy',
            'guid': current_process['guid']
        }), 409

    data = request.get_json()
    command = data.get('command')
    
    if not command:
        return jsonify({'error': 'No command provided'}), 400

    process_guid = str(uuid.uuid4())
    
    try:
        # Change to the command directory
        original_dir = os.getcwd()
        os.chdir(command)
        
        pilot = Pilot("run.ipynb", step_timeout=2)
        
        process_info = {
            'status': 'running',
            'command': command,
            'guid': process_guid,
            'payload': None
        }
        
        process_manager.set_current_process(process_info)
        process_manager.add_process(process_guid, process_info)
        
        def run_notebook_task():
            try:
                result = pilot.run_workbook()
                process_info.update({
                    'status': 'finished',
                    'payload': result
                })
            except Exception as e:
                process_info.update({
                    'status': 'error',
                    'error': str(e)
                })
            finally:
                # Revert to the original directory
                os.chdir(original_dir)
                process_manager.set_current_process(None)
                process_manager.add_process(process_guid, process_info)
                print(f"Process {process_guid} completed with status: {process_info['status']}")  # Debugging
                
        threading.Thread(target=run_notebook_task).start()
        
        return jsonify({
            'status': 'running',
            'guid': process_guid
        })
        
    except Exception as e:
        error_info = {
            'status': 'error',
            'command': command,
            'guid': process_guid,
            'error': str(e)
        }
        process_manager.add_process(process_guid, error_info)
        return jsonify({'error': str(e)}), 500

@app.route('/status', methods=['POST'])
def get_status():
    # Try to get GUID from different locations
    guid = None
    
    # Check JSON body first
    if request.is_json:
        data = request.get_json()
        guid = data.get('guid')
    
    # If no GUID in body, check query parameters
    if not guid:
        guid = request.args.get('guid')
    
    # If still no GUID, try form data
    if not guid and request.form:
        guid = request.form.get('guid')

    if not guid:
        return jsonify({
            'error': 'No GUID provided. Send GUID either in request body or as query parameter',
            'request_body': str(request.get_data()),
            'query_params': dict(request.args),
            'content_type': request.content_type
        }), 400

    # Trim whitespace from the GUID
    guid = guid.strip()
    print(f"Looking up GUID: '{guid}'")  # Debugging

    process_info = process_manager.get_process(guid)
    
    if not process_info:
        print(f"GUID not found: '{guid}'")  # Debugging
        print(f"Available GUIDs: {list(process_manager.processes.keys())}")  # Debugging
        return jsonify({'error': 'GUID unknown'}), 404
        
    return jsonify(process_info)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.177:5000
Press CTRL+C to quit
192.168.2.177 - - [30/Jan/2025 23:20:58] "POST /run HTTP/1.1" 200 -


Added process with GUID: b737c95a-76e0-486c-ba35-a69a188965ca
Processes in manager: OrderedDict({'b737c95a-76e0-486c-ba35-a69a188965ca': {'status': 'running', 'command': 'odoo-onshape-draft', 'guid': 'b737c95a-76e0-486c-ba35-a69a188965ca', 'payload': None}})
Staring step: 0
found: 1
Staring step: 0
found: 1


192.168.2.177 - - [30/Jan/2025 23:21:01] "POST /status HTTP/1.1" 200 -


Looking up GUID: 'b737c95a-76e0-486c-ba35-a69a188965ca'
Staring step: 1
Staring step: 2
found: 1


...

Looking up GUID: '5fdeb2b1-caf3-4aab-9774-d9227b95950e'
Staring step: 3
try: 0/30
try: 1/30
try: 2/30
try: 3/30
try: 4/30
try: 5/30
try: 6/30
try: 7/30
try: 8/30
try: 9/30
try: 10/30
try: 11/30
try: 12/30
try: 13/30
try: 14/30
try: 15/30
try: 16/30
try: 17/30
try: 18/30


192.168.2.177 - - [31/Jan/2025 11:43:34] "POST /status HTTP/1.1" 200 -


Looking up GUID: '5fdeb2b1-caf3-4aab-9774-d9227b95950e'
try: 19/30
try: 20/30
try: 21/30
try: 22/30
try: 23/30
try: 24/30
try: 25/30
try: 26/30
try: 27/30
try: 28/30
Could not locate the image (highest confidence = 0.474)
Staring step: 2
found: 1
Staring step: 0
found: 1
Added process with GUID: 5fdeb2b1-caf3-4aab-9774-d9227b95950e
Processes in manager: OrderedDict({'b737c95a-76e0-486c-ba35-a69a188965ca': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': 'b737c95a-76e0-486c-ba35-a69a188965ca', 'payload': None}, '438434da-7700-41a4-a41c-7f608b55c04d': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': '438434da-7700-41a4-a41c-7f608b55c04d', 'payload': None}, 'a1088465-e4ee-44a2-a43c-c3a570140eb5': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': 'a1088465-e4ee-44a2-a43c-c3a570140eb5', 'payload': None}, '8dfcb2d0-d285-4448-9edf-9ff9ea0c8613': {'status': 'finished', 'command': 'odoo-onshape-draft', 'guid': '8dfcb2d0-d285-4448-9edf-9ff9ea0c8613', '

192.168.2.177 - - [31/Jan/2025 11:44:04] "POST /status HTTP/1.1" 200 -


Looking up GUID: '5fdeb2b1-caf3-4aab-9774-d9227b95950e'
